In [14]:
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html
import util.util as util
import argparse
import os
from util import util
import torch
import models
import data
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import importlib
import torch.utils.data
from data.base_dataset import BaseDataset
import random
from torch.utils.data import Sampler
import os.path as osp
from UDAsbs.utils.data.sampler import RandomMultipleGallerySampler
from torch.utils.data import DataLoader
from UDAsbs.utils.data.preprocessor import Preprocessor
from UDAsbs.utils.data import IterLoader
from UDAsbs.utils.data import transforms as T
from UDAsbs import datasets
class Options:
    def __init__(self):
        self.CUT_mode = 'CUT'
        self.batch_size = 3
        self.checkpoints_dir = './checkpoints'
        self.crop_size = 256
        self.dataroot = '/home/jun/ReID_Dataset/'
        self.dataset_mode = 'labeled'
        self.direction = 'AtoB'
        self.display_winsize = 256
        self.easy_label = 'market2duke'
        self.epoch = 'latest'
        self.eval = False
        self.flip_equivariance = False
        self.gpu_ids = [0]
        self.init_gain = 0.02
        self.init_type = 'xavier'
        self.input_nc = 3
        self.isTrain = False
        self.lambda_GAN = 1.0
        self.lambda_NCE = 1.0
        self.load_size = 256
        self.max_dataset_size = float("inf")
        self.model = 'cut'
        self.n_layers_D = 3
        self.name = 'market2duke_segcut'
        self.nce_T = 0.07
        self.nce_idt = True
        self.nce_includes_all_negatives_from_minibatch = False
        self.nce_layers = '0,4,8,12,16'
        self.ndf = 64
        self.netD = 'basic'
        self.netF = 'mlp_sample'
        self.netF_nc = 256
        self.netG = 'resnet_9blocks'
        self.ngf = 64
        self.no_antialias = False
        self.no_antialias_up = False
        self.no_dropout = True
        self.no_flip = False
        self.normD = 'instance'
        self.normG = 'instance'
        self.num_patches = 256
        self.num_test = 50
        self.num_threads = 4
        self.output_nc = 3
        self.phase = 'test'
        self.pool_size = 0
        self.preprocess = 'resize_and_crop'
        self.random_scale_max = 3.0
        self.results_dir = './results/'
        self.serial_batches = True
        self.stylegan2_G_num_downsampling = 1
        self.suffix = ''
        self.verbose = False

opt = Options()

In [15]:
def get_data(data_dir, load_size, crop_size, batch_size=4, workers=16, num_instances=2, iters=3000):
    A = 'market1501'
    B = 'dukemtmc'
    root = osp.join(data_dir)
    dataset_A = datasets.create(A, root)
    train_A = dataset_A.train
    dataset_B = datasets.create(B, root)
    train_B = dataset_B.train

    train_transformer = T.Compose([
            T.Resize((load_size,load_size), Image.BICUBIC),
            T.RandomCrop((crop_size, crop_size)),
            T.ToTensor(),
            T.Normalize(mean=[0.5, 0.5, 0.5],
                            std=[0.5, 0.5, 0.5])
            #T.RandomErasing(probability=0.5, mean=[0.485, 0.456, 0.406])
        ])


    rmgs_flag = num_instances > 0
    if rmgs_flag:
        sampler_A = RandomMultipleGallerySampler(train_A, num_instances)
        sampler_B = RandomMultipleGallerySampler(train_B, num_instances)
        #sample = None
    else:
        sampler = None

    A_loader = IterLoader(DataLoader(Preprocessor(train_A, root=dataset_A.images_dir,
                                        transform=train_transformer),
                            batch_size=batch_size, num_workers=workers, sampler=sampler_A,
                            shuffle=not rmgs_flag, pin_memory=True, drop_last=True))

    B_loader = IterLoader(DataLoader(Preprocessor(train_B, root=dataset_B.images_dir,
                                        transform=train_transformer),
                            batch_size=batch_size, num_workers=workers, sampler=sampler_B,
                            shuffle=not rmgs_flag, pin_memory=True, drop_last=True))

    return A_loader, B_loader

In [16]:
datasetA, datasetB = get_data(opt.dataroot, opt.load_size, opt.crop_size, batch_size=4, workers=16, num_instances=2, iters=3000)

=> Market1501 loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
This dataset has been downloaded.
=> DukeMTMC-reID loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   702 |    16522 |         8
  query    |   702 |     2228 |         8
  gallery  |  1110 |    17661 |         8
  ----------------------------------------


In [30]:
data = datasetA.next()
# Tensor, Path, Person_ID, Cam_ID, Image_ID


IndexError: list index out of range

In [2]:
dataset = create_dataset(opt) 


dataset [LabeledDataset] was created


KeyboardInterrupt: 

In [ ]:
for data in dataset:
    print(data['A_id'], data['B_id'])

tensor([ 40,  40, 130]) tensor([755, 755, 869])
tensor([352, 352, 158]) tensor([1136, 1136,  911])
tensor([ 0,  0, 38]) tensor([702, 702, 753])
tensor([432, 432, 517]) tensor([1284, 1283, 1410])
tensor([ 18,  18, 412]) tensor([ 729,  731, 1251])
tensor([682, 682, 252]) tensor([ 849,  849, 1024])
tensor([275, 275, 488]) tensor([1052, 1054, 1360])
tensor([350, 350, 595]) tensor([1134, 1135,  772])
tensor([ 35,  35, 338]) tensor([ 750,  750, 1121])
tensor([219, 219, 272]) tensor([ 994,  992, 1046])
tensor([399, 399, 140]) tensor([1225, 1224,  881])
tensor([503, 503, 159]) tensor([1391, 1390,  913])
tensor([106, 106, 282]) tensor([ 842,  842, 1063])
tensor([330, 330, 151]) tensor([1113, 1113,  896])
tensor([374, 374,  48]) tensor([1183, 1182,  765])
tensor([286, 286, 289]) tensor([1068, 1067, 1069])
tensor([ 71,  71, 486]) tensor([ 799,  798, 1358])
tensor([372, 372, 638]) tensor([1180, 1181,  816])
tensor([401, 401, 490]) tensor([1226, 1227, 1363])
tensor([ 50,  50, 544]) tensor([765, 765

KeyboardInterrupt: 

{'A': tensor([[[[-0.2549, -0.2706, -0.3020,  ..., -0.1373, -0.1373, -0.1373],
           [-0.3961, -0.4039, -0.4353,  ..., -0.1294, -0.1294, -0.1294],
           [-0.5373, -0.5451, -0.5686,  ..., -0.1216, -0.1216, -0.1216],
           ...,
           [ 0.4510,  0.4510,  0.4431,  ...,  0.3647,  0.3647,  0.3647],
           [ 0.4431,  0.4353,  0.4275,  ...,  0.3647,  0.3647,  0.3647],
           [ 0.4275,  0.4275,  0.4196,  ...,  0.3725,  0.3725,  0.3725]],
 
          [[-0.3098, -0.3255, -0.3569,  ..., -0.1608, -0.1608, -0.1608],
           [-0.4510, -0.4588, -0.4902,  ..., -0.1529, -0.1529, -0.1529],
           [-0.5922, -0.6000, -0.6235,  ..., -0.1451, -0.1451, -0.1451],
           ...,
           [ 0.3569,  0.3569,  0.3490,  ...,  0.2549,  0.2549,  0.2549],
           [ 0.3490,  0.3412,  0.3333,  ...,  0.2549,  0.2549,  0.2549],
           [ 0.3333,  0.3333,  0.3255,  ...,  0.2627,  0.2627,  0.2627]],
 
          [[-0.3569, -0.3725, -0.4039,  ..., -0.2000, -0.2000, -0.2000],
        

In [ ]:
# Function to convert tensor to image for displaying
def tensor_to_image(tensor):
    tensor = tensor.clone().detach().cpu()  # Clone the tensor to avoid modifying the original and move to CPU
    tensor = F.interpolate(tensor, size=[256,128], mode='bilinear', align_corners=False)
    tensor = tensor.squeeze(0)  # Remove batch dimension
    tensor = tensor * torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1) + torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1)  # De-normalize
    tensor = tensor.permute(1, 2, 0)  # Change from CxHxW to HxWxC
    tensor = tensor.numpy()  # Convert to numpy array
    tensor = (tensor * 255).astype(np.uint8)  # Convert to uint8
    return tensor

crop_size_w = 256
crop_size_h = 256
# Dataset loader
transform = transforms.Compose([
    transforms.Resize((crop_size_h, crop_size_w)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
import os
from data import create_dataset
from models import create_model
from util.visualizer import save_images
from util import html
import util.util as util
import argparse
import os
from util import util
import torch
import models
import data
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
class Options:
    def __init__(self):
        self.CUT_mode = 'CUT'
        self.batch_size = 1
        self.checkpoints_dir = './checkpoints'
        self.crop_size = 256
        self.dataroot = '/home/jun/CUT/datasets/market2duke'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.display_winsize = 256
        self.easy_label = 'market2duke'
        self.epoch = 'latest'
        self.eval = False
        self.flip_equivariance = False
        self.gpu_ids = [0]
        self.init_gain = 0.02
        self.init_type = 'xavier'
        self.input_nc = 3
        self.isTrain = False
        self.lambda_GAN = 1.0
        self.lambda_NCE = 1.0
        self.load_size = 256
        self.max_dataset_size = float("inf")
        self.model = 'cut'
        self.n_layers_D = 3
        self.name = 'market2duke_segcut'
        self.nce_T = 0.07
        self.nce_idt = True
        self.nce_includes_all_negatives_from_minibatch = False
        self.nce_layers = '0,4,8,12,16'
        self.ndf = 64
        self.netD = 'basic'
        self.netF = 'mlp_sample'
        self.netF_nc = 256
        self.netG = 'resnet_9blocks'
        self.ngf = 64
        self.no_antialias = False
        self.no_antialias_up = False
        self.no_dropout = True
        self.no_flip = False
        self.normD = 'instance'
        self.normG = 'instance'
        self.num_patches = 256
        self.num_test = 50
        self.num_threads = 4
        self.output_nc = 3
        self.phase = 'test'
        self.pool_size = 0
        self.preprocess = 'resize_and_crop'
        self.random_scale_max = 3.0
        self.results_dir = './results/'
        self.serial_batches = False
        self.stylegan2_G_num_downsampling = 1
        self.suffix = ''
        self.verbose = False

opt = Options()
#dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
train_dataset = create_dataset(util.copyconf(opt, phase="train"))
model = create_model(opt)      # create a model given opt.model and other options
model.load_networks('latest')
# create a webpage for viewing the results
#web_dir = os.path.join(opt.results_dir, opt.name, '{}_{}'.format(opt.phase, opt.epoch))  # define the website directory
#print('creating web directory', web_dir)

dataset [UnalignedDataset] was created
model [CUTModel] was created
loading the model from ./checkpoints/market2duke_segcut/latest_net_G.pth
